In [1]:
import pandas as pd
import mysql.connector

# Step 1: Load the Excel data
file_path = r"C:\Users\CHAKU FOODS\Documents\Farm Table WKT.xlsx"
data = pd.read_excel(file_path)

# Step 2: Establish connection to the MySQL database
try:
    conn = mysql.connector.connect(
        host='localhost',  # Change as necessary
        user='root',  # Replace with your MySQL username
        password='KpakpoParker700!',  # Replace with your MySQL password
        database='chaku_foods'  # Replace with your database name
    )
    cursor = conn.cursor()
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit(1)

# Step 3: Iterate through the Excel data and update the MySQL farm table
for index, row in data.iterrows():
    try:
        # Extracting relevant data
        last_name = row['last_name']
        first_name = row['first_name']
        farm_number = row['farm_number']
        geo_coordinates = row['geo_coordinates']  # Should be in WKT format (e.g., 'POINT(lon lat)')
        geo_boundaries = row['geo_boundaries']    # Should be in WKT format (e.g., 'POLYGON((x1 y1, x2 y2, ...))')

        # Fetching farmer_id using last_name and first_name
        cursor.execute(
            """
            SELECT farmer_id FROM farmer WHERE last_name = %s AND first_name = %s
            """,
            (last_name, first_name)
        )
        farmer_result = cursor.fetchone()

        if farmer_result:
            farmer_id = farmer_result[0]

            # Fetching farm_id using farmer_id and farm_number
            cursor.execute(
                """
                SELECT farm_id FROM farm WHERE farmer_id = %s AND farm_number = %s
                """,
                (farmer_id, farm_number)
            )
            farm_result = cursor.fetchone()

            if farm_result:
                farm_id = farm_result[0]

                # Updating the geo_coordinates and geo_boundaries columns in the farm table
                update_query = """
                    UPDATE farm 
                    SET geo_coordinates = ST_GeomFromText(%s), geo_boundaries = ST_GeomFromText(%s)
                    WHERE farm_id = %s
                """
                cursor.execute(update_query, (geo_coordinates, geo_boundaries, farm_id))
        
    except mysql.connector.Error as err:
        print(f"Error updating record at row {index}: {err}")

# Step 4: Commit the changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("Farm table updated successfully.")


Farm table updated successfully.
